# 📘 Step 1: MLflow Tracking - 실험 추적 기초

## 목표
- MLflow Tracking의 기본 개념 이해
- Parameters, Metrics, Artifacts 로깅
- MLflow UI에서 실험 결과 확인

## 학습 포인트
- `mlflow.start_run()` 사용법
- `log_param()`, `log_metric()`, `log_artifact()` 차이
- Run, Experiment 개념
- 여러 실험 비교 및 분석

## 0. 사전 준비

### ⚠️ 주의사항
이 노트북을 실행하기 전에 **Step 0**을 먼저 완료해야 합니다.
- `00_setup_baseline_rag_agent.ipynb` 실행 완료
- `.env` 파일 설정 완료
- 필요한 패키지 설치 완료

## 1. 환경 설정 및 패키지 임포트

### 1.1 환경 변수 로드

In [9]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

True

### 1.2 필요한 라이브러리 임포트

In [10]:
import time
import json
from typing import TypedDict, List
from datetime import datetime

# MLflow
import mlflow
from mlflow import MlflowClient

# LangChain & LangGraph
from langchain_aws import ChatBedrock, BedrockEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END

print("✅ 라이브러리 임포트 완료")
print(f"MLflow 버전: {mlflow.__version__}")

✅ 라이브러리 임포트 완료
MLflow 버전: 3.7.0


## 2. MLflow Tracking 설정

### 2.1 Tracking URI 설정

In [11]:
# 로컬 디렉토리에 MLflow 데이터 저장
mlflow.set_tracking_uri("./mlruns")

print("✅ MLflow Tracking URI 설정 완료")
print(f"📁 Tracking URI: {mlflow.get_tracking_uri()}")

✅ MLflow Tracking URI 설정 완료
📁 Tracking URI: ./mlruns


### 2.2 Experiment 생성

**Experiment**: 관련된 여러 Run을 그룹화하는 단위

In [12]:
# Experiment 이름 설정
experiment_name = "rag_agent_experiments"

# Experiment 생성 (이미 있으면 기존 것 사용)
mlflow.set_experiment(experiment_name)

# Experiment 정보 확인
experiment = mlflow.get_experiment_by_name(experiment_name)

print("✅ Experiment 설정 완료")
print(f"📊 Experiment Name: {experiment.name}")
print(f"🆔 Experiment ID: {experiment.experiment_id}")
print(f"📁 Artifact Location: {experiment.artifact_location}")

/Users/hank_kim/Documents/taehan/mlflow-genai-tutorial/.venv/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, store_uri)
2025/12/13 00:29:19 INFO mlflow.tracking.fluent: Experiment with name 'rag_agent_experiments' does not exist. Creating a new experiment.


✅ Experiment 설정 완료
📊 Experiment Name: rag_agent_experiments
🆔 Experiment ID: 313401264484788163
📁 Artifact Location: /Users/hank_kim/Documents/taehan/mlflow-genai-tutorial/mlruns/313401264484788163


## 3. RAG Agent 재구성

Step 0에서 만든 RAG Agent를 다시 구성합니다.

### 3.1 샘플 문서 및 Vector Store 생성

In [13]:
# 샘플 문서 (Step 0과 동일)
sample_documents = [
    """
    RAG (Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)의 능력을 향상시키는 기술입니다.
    RAG는 외부 지식 베이스에서 관련 정보를 검색하고, 이를 바탕으로 더 정확하고 사실에 기반한 답변을 생성합니다.
    이 방법은 모델이 학습하지 않은 최신 정보나 특정 도메인 지식을 활용할 수 있게 해줍니다.
    """,
    """
    Vector Store는 문서의 임베딩 벡터를 저장하고 검색하는 데이터베이스입니다.
    FAISS, Chroma, Pinecone 등이 대표적인 Vector Store입니다.
    Vector Store는 유사도 검색(Similarity Search)을 통해 쿼리와 가장 관련성 높은 문서를 빠르게 찾아냅니다.
    """,
    """
    LangGraph는 LangChain 위에 구축된 상태 기반 그래프 프레임워크입니다.
    복잡한 AI 에이전트 워크플로우를 명확한 상태와 노드로 표현할 수 있습니다.
    각 노드는 특정 작업을 수행하고, 엣지는 노드 간의 전환 조건을 정의합니다.
    """,
    """
    Embeddings는 텍스트를 고차원 벡터 공간의 점으로 표현하는 기술입니다.
    OpenAI의 text-embedding-3-small, text-embedding-3-large 모델이 널리 사용됩니다.
    좋은 임베딩은 의미적으로 유사한 텍스트를 벡터 공간에서 가까이 위치시킵니다.
    """,
    """
    Chunking은 긴 문서를 작은 조각으로 나누는 과정입니다.
    적절한 chunk 크기는 검색 정확도와 컨텍스트 양의 균형을 맞추는 데 중요합니다.
    일반적으로 512~1024 토큰 크기의 chunk가 많이 사용되며, 문장 경계를 고려한 분할이 효과적입니다.
    """,
]

print(f"✅ {len(sample_documents)}개의 샘플 문서 준비 완료")

✅ 5개의 샘플 문서 준비 완료


In [14]:
def create_vector_store(chunk_size: int = 512, chunk_overlap: int = 50):
    """
    Vector Store를 생성하는 함수
    
    Args:
        chunk_size: 텍스트를 나눌 크기
        chunk_overlap: chunk 간 중복 크기
    """
    # 텍스트 분할기
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    
    # 문서 분할
    documents = [Document(page_content=doc.strip()) for doc in sample_documents]
    split_docs = text_splitter.split_documents(documents)
    
    # 임베딩 모델 초기화
    embeddings = BedrockEmbeddings(
        model_id=os.environ["AWS_EMD_MODEL_ID"],
        aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
        aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
        region_name=os.environ["AWS_REGION"],
    )
    
    # Vector Store 생성
    vector_store = FAISS.from_documents(split_docs, embeddings)
    
    return vector_store, len(split_docs)

print("✅ create_vector_store 함수 정의 완료")

✅ create_vector_store 함수 정의 완료


### 3.2 RAG State 및 Nodes 정의

In [15]:
class RAGState(TypedDict):
    """RAG Agent의 상태"""
    query: str
    retrieved_documents: List[Document]
    context: str
    answer: str
    metadata: dict

print("✅ RAGState 정의 완료")

✅ RAGState 정의 완료


In [16]:
def create_rag_agent(vector_store, top_k: int = 3, llm_model: str = "gpt-4o-mini", temperature: float = 0.7):
    """
    RAG Agent를 생성하는 함수
    
    Args:
        vector_store: FAISS vector store
        top_k: 검색할 문서 개수
        llm_model: 사용할 LLM 모델
        temperature: LLM temperature
    """
    
    # Retriever Node
    def retriever_node(state: RAGState) -> RAGState:
        start_time = time.time()
        
        retrieved_docs = vector_store.similarity_search(state["query"], k=top_k)
        
        state["retrieved_documents"] = retrieved_docs
        state["metadata"]["retrieval_time"] = time.time() - start_time
        state["metadata"]["num_retrieved_docs"] = len(retrieved_docs)
        
        return state
    
    # Generator Node
    def generator_node(state: RAGState) -> RAGState:
        start_time = time.time()
        
        # Context 생성
        context = "\n\n".join([
            f"[문서 {i+1}]\n{doc.page_content}"
            for i, doc in enumerate(state["retrieved_documents"])
        ])
        state["context"] = context
        
        # Prompt 템플릿
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", "당신은 질문에 답변하는 친절한 AI 어시스턴트입니다. 제공된 컨텍스트를 기반으로 정확하고 간결한 답변을 제공하세요."),
            ("human", "컨텍스트:\n{context}\n\n질문: {query}\n\n답변:")
        ])
        
        # LLM 초기화
        llm = ChatBedrock(
            model_id=os.environ["AWS_MODEL_ID"],
            aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
            aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
            region_name=os.environ["AWS_REGION"],
            temperature=0.0,
        )
        
        # 답변 생성
        messages = prompt_template.format_messages(context=context, query=state["query"])
        response = llm.invoke(messages)
        
        state["answer"] = response.content
        state["metadata"]["generation_time"] = time.time() - start_time
        
        return state
    
    # Graph 구성
    workflow = StateGraph(RAGState)
    workflow.add_node("retriever", retriever_node)
    workflow.add_node("generator", generator_node)
    workflow.add_edge(START, "retriever")
    workflow.add_edge("retriever", "generator")
    workflow.add_edge("generator", END)
    
    return workflow.compile()

print("✅ create_rag_agent 함수 정의 완료")

✅ create_rag_agent 함수 정의 완료


## 4. MLflow로 첫 번째 Run 기록하기

### 4.1 기본 Run 생성

In [17]:
print("="*80)
print("🚀 첫 번째 MLflow Run 시작")
print("="*80)

# MLflow Run 시작
with mlflow.start_run(run_name="baseline_rag_v1") as run:
    
    # 1. Parameters 로깅
    chunk_size = 512
    chunk_overlap = 50
    top_k = 3
    llm_model = os.environ["AWS_MODEL_ID"]
    temperature = 0.0
    emd_model = os.environ["AWS_EMD_MODEL_ID"]
    
    mlflow.log_param("chunk_size", chunk_size)
    mlflow.log_param("chunk_overlap", chunk_overlap)
    mlflow.log_param("top_k", top_k)
    mlflow.log_param("llm_model", llm_model)
    mlflow.log_param("temperature", temperature)
    mlflow.log_param("embedding_model", emd_model)
    
    print("✅ Parameters 로깅 완료")
    
    # 2. Vector Store 및 RAG Agent 생성
    vector_store, num_chunks = create_vector_store(chunk_size, chunk_overlap)
    rag_agent = create_rag_agent(vector_store, top_k, llm_model, temperature)
    
    mlflow.log_param("num_chunks", num_chunks)
    print(f"✅ Vector Store 생성 완료 (총 {num_chunks}개 chunks)")
    
    # 3. 테스트 쿼리 실행
    test_query = "RAG가 무엇인가요?"
    
    initial_state = {
        "query": test_query,
        "retrieved_documents": [],
        "context": "",
        "answer": "",
        "metadata": {}
    }
    
    overall_start = time.time()
    result = rag_agent.invoke(initial_state)
    overall_time = time.time() - overall_start
    
    print(f"\n✅ RAG Agent 실행 완료")
    print(f"💬 답변: {result['answer'][:100]}...")
    
    # 4. Metrics 로깅
    mlflow.log_metric("overall_time", overall_time)
    mlflow.log_metric("retrieval_time", result['metadata']['retrieval_time'])
    mlflow.log_metric("generation_time", result['metadata']['generation_time'])
    mlflow.log_metric("num_retrieved_docs", result['metadata']['num_retrieved_docs'])
    mlflow.log_metric("answer_length", len(result['answer']))
    
    print("✅ Metrics 로깅 완료")
    
    # 5. Artifacts 로깅
    # 5.1 답변을 텍스트 파일로 저장
    mlflow.log_text(result['answer'], "output_answer.txt")
    
    # 5.2 검색된 문서를 JSON으로 저장
    retrieved_docs_data = [
        {"index": i, "content": doc.page_content}
        for i, doc in enumerate(result['retrieved_documents'])
    ]
    mlflow.log_dict(retrieved_docs_data, "retrieved_documents.json")
    
    # 5.3 실행 정보를 JSON으로 저장
    run_info = {
        "query": test_query,
        "answer": result['answer'],
        "metrics": result['metadata'],
        "parameters": {
            "chunk_size": chunk_size,
            "top_k": top_k,
            "llm_model": llm_model
        }
    }
    mlflow.log_dict(run_info, "run_summary.json")
    
    print("✅ Artifacts 로깅 완료")
    
    # Run 정보 출력
    print(f"\n{'='*80}")
    print("📊 Run 정보")
    print(f"{'='*80}")
    print(f"🆔 Run ID: {run.info.run_id}")
    print(f"📊 Experiment ID: {run.info.experiment_id}")
    print(f"📁 Artifact URI: {run.info.artifact_uri}")
    print(f"⏱️  실행 시간: {overall_time:.3f}초")
    print(f"{'='*80}")

🚀 첫 번째 MLflow Run 시작
✅ Parameters 로깅 완료
✅ Vector Store 생성 완료 (총 5개 chunks)

✅ RAG Agent 실행 완료
💬 답변: RAG(Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)의 능력을 향상시키는 기술입니다.

**주요 특징:**
- 외부 지식 베이스에서 관련 정...
✅ Metrics 로깅 완료
✅ Artifacts 로깅 완료

📊 Run 정보
🆔 Run ID: d8bdf68c0679464ea9f09fdbba644e08
📊 Experiment ID: 313401264484788163
📁 Artifact URI: /Users/hank_kim/Documents/taehan/mlflow-genai-tutorial/mlruns/313401264484788163/d8bdf68c0679464ea9f09fdbba644e08/artifacts
⏱️  실행 시간: 5.057초


## 5. 여러 하이퍼파라미터 조합 실험

다양한 파라미터 조합으로 여러 Run을 생성하여 비교합니다.

### 5.1 실험 설정

In [18]:
# 실험할 파라미터 조합
experiment_configs = [
    {"name": "small_chunks_low_k", "chunk_size": 256, "top_k": 3, "temperature": 0.7},
    {"name": "medium_chunks_medium_k", "chunk_size": 512, "top_k": 5, "temperature": 0.7},
    {"name": "large_chunks_high_k", "chunk_size": 1024, "top_k": 10, "temperature": 0.7},
    {"name": "medium_chunks_low_temp", "chunk_size": 512, "top_k": 3, "temperature": 0.3},
    {"name": "medium_chunks_high_temp", "chunk_size": 512, "top_k": 3, "temperature": 1.0},
]

# 테스트할 쿼리들
test_queries = [
    "RAG가 무엇인가요?",
    "Vector Store의 역할은 무엇인가요?",
]

print(f"✅ {len(experiment_configs)}개의 실험 설정 준비 완료")
print(f"✅ {len(test_queries)}개의 테스트 쿼리 준비 완료")

✅ 5개의 실험 설정 준비 완료
✅ 2개의 테스트 쿼리 준비 완료


### 5.2 배치 실험 실행

In [19]:
print("\n" + "="*80)
print("🔬 배치 실험 시작")
print("="*80)

all_run_ids = []

for idx, config in enumerate(experiment_configs, 1):
    print(f"\n{'='*80}")
    print(f"실험 {idx}/{len(experiment_configs)}: {config['name']}")
    print(f"{'='*80}")
    
    with mlflow.start_run(run_name=config['name']) as run:
        llm_model = os.environ["AWS_MODEL_ID"]
        emd_model = os.environ["AWS_EMD_MODEL_ID"]
        # Parameters 로깅
        mlflow.log_param("chunk_size", config['chunk_size'])
        mlflow.log_param("chunk_overlap", 50)
        mlflow.log_param("top_k", config['top_k'])
        mlflow.log_param("llm_model", llm_model)
        mlflow.log_param("temperature", config['temperature'])
        mlflow.log_param("embedding_model", emd_model)
        
        # Vector Store 및 RAG Agent 생성
        vector_store, num_chunks = create_vector_store(
            chunk_size=config['chunk_size'],
            chunk_overlap=50
        )
        mlflow.log_param("num_chunks", num_chunks)
        
        rag_agent = create_rag_agent(
            vector_store=vector_store,
            top_k=config['top_k'],
            llm_model="gpt-4o-mini",
            temperature=config['temperature']
        )
        
        # 여러 쿼리 테스트
        total_time = 0
        total_retrieval_time = 0
        total_generation_time = 0
        
        for query_idx, query in enumerate(test_queries):
            initial_state = {
                "query": query,
                "retrieved_documents": [],
                "context": "",
                "answer": "",
                "metadata": {}
            }
            
            start_time = time.time()
            result = rag_agent.invoke(initial_state)
            elapsed_time = time.time() - start_time
            
            total_time += elapsed_time
            total_retrieval_time += result['metadata']['retrieval_time']
            total_generation_time += result['metadata']['generation_time']
            
            # 쿼리별 메트릭 로깅
            mlflow.log_metric(f"query_{query_idx}_time", elapsed_time)
            mlflow.log_metric(f"query_{query_idx}_answer_length", len(result['answer']))
        
        # 평균 메트릭 계산 및 로깅
        num_queries = len(test_queries)
        mlflow.log_metric("avg_overall_time", total_time / num_queries)
        mlflow.log_metric("avg_retrieval_time", total_retrieval_time / num_queries)
        mlflow.log_metric("avg_generation_time", total_generation_time / num_queries)
        mlflow.log_metric("total_time", total_time)
        
        # Run ID 저장
        all_run_ids.append(run.info.run_id)
        
        print(f"✅ 완료 - 평균 실행 시간: {total_time / num_queries:.3f}초")
        print(f"🆔 Run ID: {run.info.run_id}")

print(f"\n{'='*80}")
print(f"✅ 총 {len(all_run_ids)}개의 실험 완료")
print(f"{'='*80}")


🔬 배치 실험 시작

실험 1/5: small_chunks_low_k
✅ 완료 - 평균 실행 시간: 4.935초
🆔 Run ID: 98b1875988ee4fd4b47151de5b6d550c

실험 2/5: medium_chunks_medium_k
✅ 완료 - 평균 실행 시간: 5.811초
🆔 Run ID: 071788b87b1249c3adda7d45e2b3d4df

실험 3/5: large_chunks_high_k
✅ 완료 - 평균 실행 시간: 5.908초
🆔 Run ID: 2e145713bb344f3098c3d54b62798521

실험 4/5: medium_chunks_low_temp
✅ 완료 - 평균 실행 시간: 5.101초
🆔 Run ID: 6babc51e59a740b9af29c9ac309178a3

실험 5/5: medium_chunks_high_temp
✅ 완료 - 평균 실행 시간: 4.860초
🆔 Run ID: 84ef7495b61a44d49cbcc90965479969

✅ 총 5개의 실험 완료


## 6. MLflow UI에서 결과 확인

### 6.1 MLflow UI 실행 방법

터미널에서 다음 명령어를 실행하세요:

```bash
mlflow ui --port 5000
```

또는 다른 포트 사용:

```bash
mlflow ui --port 8080
```

그런 다음 브라우저에서 `http://localhost:5000` (또는 설정한 포트)로 접속하세요.

### 6.2 UI에서 확인할 내용

1. **Experiments 페이지**
   - `rag_agent_experiments` 실험 선택
   - 모든 Run 목록 확인

2. **Run 비교**
   - 여러 Run 선택 (체크박스)
   - "Compare" 버튼 클릭
   - Parameters, Metrics 비교

3. **차트 시각화**
   - Parallel Coordinates Plot
   - Scatter Plot (Parameters vs Metrics)
   - Box Plot

4. **개별 Run 상세 정보**
   - Parameters 탭
   - Metrics 탭
   - Artifacts 탭 (생성된 파일들)

## 7. Python API로 Run 정보 조회하기

### 7.1 MlflowClient로 Run 검색

In [20]:
# MlflowClient 초기화
client = MlflowClient()

# 현재 실험의 모든 Run 조회
experiment = mlflow.get_experiment_by_name("rag_agent_experiments")
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.avg_overall_time ASC"]  # 평균 실행 시간 기준 정렬
)

print(f"\n📊 총 {len(runs)}개의 Run 발견\n")
print("="*80)
print(f"{'Run Name':<30} {'Avg Time':<12} {'Top K':<8} {'Chunk Size':<12}")
print("="*80)

for run in runs[:10]:  # 최대 10개만 출력
    run_name = run.data.tags.get('mlflow.runName', 'N/A')
    avg_time = run.data.metrics.get('avg_overall_time', 0)
    top_k = run.data.params.get('top_k', 'N/A')
    chunk_size = run.data.params.get('chunk_size', 'N/A')
    
    print(f"{run_name:<30} {avg_time:<12.3f} {top_k:<8} {chunk_size:<12}")

print("="*80)


📊 총 6개의 Run 발견

Run Name                       Avg Time     Top K    Chunk Size  
medium_chunks_high_temp        4.860        3        512         
small_chunks_low_k             4.935        3        256         
medium_chunks_low_temp         5.101        3        512         
medium_chunks_medium_k         5.811        5        512         
large_chunks_high_k            5.908        10       1024        
baseline_rag_v1                0.000        3        512         


### 7.2 최고 성능 Run 찾기

In [21]:
# 가장 빠른 실행 시간을 가진 Run 찾기
if runs:
    best_run = runs[0]  # 이미 avg_overall_time으로 정렬됨
    
    print("\n" + "="*80)
    print("🏆 최고 성능 Run")
    print("="*80)
    
    print(f"\n🆔 Run ID: {best_run.info.run_id}")
    print(f"📝 Run Name: {best_run.data.tags.get('mlflow.runName', 'N/A')}")
    
    print("\n📊 Parameters:")
    for key, value in best_run.data.params.items():
        print(f"  • {key}: {value}")
    
    print("\n📈 Metrics:")
    for key, value in best_run.data.metrics.items():
        if 'time' in key:
            print(f"  • {key}: {value:.3f}초")
        else:
            print(f"  • {key}: {value:.2f}")
    
    print("\n" + "="*80)


🏆 최고 성능 Run

🆔 Run ID: 84ef7495b61a44d49cbcc90965479969
📝 Run Name: medium_chunks_high_temp

📊 Parameters:
  • chunk_overlap: 50
  • num_chunks: 5
  • top_k: 3
  • chunk_size: 512
  • embedding_model: amazon.titan-embed-text-v2:0
  • temperature: 1.0
  • llm_model: global.anthropic.claude-sonnet-4-5-20250929-v1:0

📈 Metrics:
  • query_0_time: 4.501초
  • avg_overall_time: 4.860초
  • query_1_time: 5.220초
  • total_time: 9.721초
  • query_0_answer_length: 281.00
  • query_1_answer_length: 315.00
  • avg_retrieval_time: 0.111초
  • avg_generation_time: 4.747초



### 7.3 Run의 Artifacts 확인

In [22]:
# 첫 번째 Run의 Artifacts 목록 조회
if runs:
    sample_run = runs[0]
    artifacts = client.list_artifacts(sample_run.info.run_id)
    
    print("\n" + "="*80)
    print(f"📁 Artifacts (Run: {sample_run.data.tags.get('mlflow.runName', 'N/A')})")
    print("="*80)
    
    for artifact in artifacts:
        print(f"  • {artifact.path} ({artifact.file_size} bytes)")
    
    print("="*80)


📁 Artifacts (Run: medium_chunks_high_temp)


## 8. 실험 결과 분석 및 시각화

### 8.1 데이터프레임으로 변환

In [23]:
import pandas as pd

# Run 데이터를 DataFrame으로 변환
data = []
for run in runs:
    row = {
        'run_id': run.info.run_id,
        'run_name': run.data.tags.get('mlflow.runName', 'N/A'),
        'chunk_size': int(run.data.params.get('chunk_size', 0)),
        'top_k': int(run.data.params.get('top_k', 0)),
        'temperature': float(run.data.params.get('temperature', 0)),
        'avg_time': run.data.metrics.get('avg_overall_time', 0),
        'avg_retrieval_time': run.data.metrics.get('avg_retrieval_time', 0),
        'avg_generation_time': run.data.metrics.get('avg_generation_time', 0),
    }
    data.append(row)

df = pd.DataFrame(data)

print("\n📊 실험 결과 요약\n")
print(df.to_string(index=False))


📊 실험 결과 요약

                          run_id                run_name  chunk_size  top_k  temperature  avg_time  avg_retrieval_time  avg_generation_time
84ef7495b61a44d49cbcc90965479969 medium_chunks_high_temp         512      3          1.0  4.860455            0.111152             4.747003
98b1875988ee4fd4b47151de5b6d550c      small_chunks_low_k         256      3          0.7  4.935054            0.125673             4.806494
6babc51e59a740b9af29c9ac309178a3  medium_chunks_low_temp         512      3          0.3  5.100586            0.121225             4.975662
071788b87b1249c3adda7d45e2b3d4df  medium_chunks_medium_k         512      5          0.7  5.810857            0.109547             5.695684
2e145713bb344f3098c3d54b62798521     large_chunks_high_k        1024     10          0.7  5.907521            0.121863             5.781771
d8bdf68c0679464ea9f09fdbba644e08         baseline_rag_v1         512      3          0.0  0.000000            0.000000             0.000000


### 8.2 간단한 분석

In [24]:
print("\n" + "="*80)
print("📈 실험 결과 분석")
print("="*80)

# chunk_size에 따른 평균 시간
print("\n1️⃣ Chunk Size별 평균 실행 시간:")
chunk_analysis = df.groupby('chunk_size')['avg_time'].mean().sort_values()
for chunk_size, avg_time in chunk_analysis.items():
    print(f"  • {chunk_size}: {avg_time:.3f}초")

# top_k에 따른 평균 시간
print("\n2️⃣ Top-K별 평균 실행 시간:")
topk_analysis = df.groupby('top_k')['avg_time'].mean().sort_values()
for top_k, avg_time in topk_analysis.items():
    print(f"  • {top_k}: {avg_time:.3f}초")

# temperature에 따른 평균 시간
print("\n3️⃣ Temperature별 평균 실행 시간:")
temp_analysis = df.groupby('temperature')['avg_time'].mean().sort_values()
for temp, avg_time in temp_analysis.items():
    print(f"  • {temp}: {avg_time:.3f}초")

# 전체 통계
print("\n4️⃣ 전체 통계:")
print(f"  • 평균 실행 시간: {df['avg_time'].mean():.3f}초 (± {df['avg_time'].std():.3f})")
print(f"  • 최소 실행 시간: {df['avg_time'].min():.3f}초")
print(f"  • 최대 실행 시간: {df['avg_time'].max():.3f}초")

print("\n" + "="*80)


📈 실험 결과 분석

1️⃣ Chunk Size별 평균 실행 시간:
  • 512: 3.943초
  • 256: 4.935초
  • 1024: 5.908초

2️⃣ Top-K별 평균 실행 시간:
  • 3: 3.724초
  • 5: 5.811초
  • 10: 5.908초

3️⃣ Temperature별 평균 실행 시간:
  • 0.0: 0.000초
  • 1.0: 4.860초
  • 0.3: 5.101초
  • 0.7: 5.551초

4️⃣ 전체 통계:
  • 평균 실행 시간: 4.436초 (± 2.218)
  • 최소 실행 시간: 0.000초
  • 최대 실행 시간: 5.908초



## 9. Run에 태그 추가하기

Run에 태그를 추가하여 분류 및 검색을 쉽게 할 수 있습니다.

In [25]:
# 최고 성능 Run에 태그 추가
if runs:
    best_run_id = runs[0].info.run_id
    
    client.set_tag(best_run_id, "performance", "best")
    client.set_tag(best_run_id, "status", "production_candidate")
    
    print(f"✅ Run {best_run_id}에 태그 추가 완료")
    print("  • performance: best")
    print("  • status: production_candidate")

✅ Run 84ef7495b61a44d49cbcc90965479969에 태그 추가 완료
  • performance: best
  • status: production_candidate


## 10. 요약 및 다음 단계

### ✅ 완료한 작업
1. MLflow Tracking URI 및 Experiment 설정
2. 첫 번째 Run 생성 및 Parameters, Metrics, Artifacts 로깅
3. 여러 하이퍼파라미터 조합으로 배치 실험 실행
4. MLflow UI에서 결과 확인 방법 학습
5. Python API로 Run 조회 및 분석

### 📊 주요 학습 내용
- **Run**: 단일 실험 실행 (Parameters + Metrics + Artifacts)
- **Experiment**: 관련된 여러 Run의 그룹
- **Parameters**: 실험 설정값 (chunk_size, top_k 등)
- **Metrics**: 측정 결과 (실행 시간, 정확도 등)
- **Artifacts**: 생성된 파일 (모델, 텍스트, JSON 등)

### 🎯 다음 단계 (Step 2)
다음 노트북에서는 **MLflow Tracing (Autolog)**을 활용하여:
- LangChain Autolog로 원라인 자동 추적
- Trace 구조 이해 (Span, Parent-Child)
- Jupyter Notebook에서 실시간 Trace 시각화
- Token usage 자동 추적

→ `02_mlflow_tracing_autolog.ipynb`로 계속하세요!

### 💡 추가 실습 아이디어
1. 다른 embedding 모델 비교 (text-embedding-3-small vs large)
2. 다양한 LLM 모델 테스트 (gpt-4o-mini vs gpt-4o)
3. 더 많은 테스트 쿼리로 평가
4. 커스텀 Metrics 추가 (답변 품질, 정확도 등)

## 📚 참고 자료

- [MLflow Tracking Documentation](https://www.mlflow.org/docs/2.21.3/tracking/)
- [MLflow Python API - Logging Functions](https://www.mlflow.org/docs/2.21.3/python_api/mlflow.html#mlflow.log_param)
- [MLflow Concepts](https://www.mlflow.org/docs/2.21.3/concepts.html)
- [MLflow Quickstart](https://www.mlflow.org/docs/2.21.3/getting-started/intro-quickstart/index.html)